# **Install Lib - Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q gdown
!pip install -q --upgrade gdown
!pip install -q monai timm torch torchvision scikit-learn tqdm matplotlib pillow pandas openpyxl
!pip install -q gspread gspread_dataframe google-auth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 48.8 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc_gs = gspread.authorize(creds)

In [ ]:
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1pn_dqQmvpUNrVzU-QwI0oxOQpShEu22KmgIQQwMtkk4/edit#gid=0"
sh = gc_gs.open_by_url(SPREADSHEET_URL)
ws = sh.worksheet("Trang tính1")

In [ ]:
import os, zipfile, gdown
os.makedirs("/content/data", exist_ok=True)

file_id = "1hqv9VWk_iu1EQAyauLFCFY7GfEInPRsC"
url = f"https://drive.google.com/uc?id={file_id}"
zip_path = "/content/Mura.zip"

if not os.path.exists("/content/data/MURA-v1.1"):
    if not os.path.exists(zip_path):
        gdown.download(url, zip_path, quiet=False)
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall("/content/data")
else:
    print("Dataset already extracted.")

print(os.listdir("/content/data"))


Downloading...
From (original): https://drive.google.com/uc?id=1hqv9VWk_iu1EQAyauLFCFY7GfEInPRsC
From (redirected): https://drive.google.com/uc?id=1hqv9VWk_iu1EQAyauLFCFY7GfEInPRsC&confirm=t&uuid=9d386e15-d299-45bc-b523-46d3bc6a5f5a
To: /content/Mura.zip
100%|██████████| 3.38G/3.38G [01:02<00:00, 53.7MB/s]


['MURA-v1.1']


In [ ]:
import os, glob, re
from collections import defaultdict

base_dir = "./data/MURA-v1.1"
body_parts = ["XR_ELBOW","XR_FINGER","XR_FOREARM","XR_HUMERUS","XR_SHOULDER","XR_HAND","XR_WRIST"]
IMG_EXTS = ("*.png","*.jpg","*.jpeg","*.bmp","*.tif","*.tiff")

pat_re   = re.compile(r"patient\d+", re.IGNORECASE)
study_re = re.compile(r"study\d+_(positive|negative)", re.IGNORECASE)

def count_patients(part, split):
    path = os.path.join(base_dir, split, part)
    if not os.path.exists(path): return 0
    return sum(1 for name in os.listdir(path)
               if os.path.isdir(os.path.join(path, name)) and pat_re.match(name))

def count_studies(part, split):
    root = os.path.join(base_dir, split, part)
    if not os.path.exists(root): return 0
    total = 0
    for p in os.listdir(root):
        pdir = os.path.join(root, p)
        if os.path.isdir(pdir) and pat_re.match(p):
            total += sum(1 for s in os.listdir(pdir)
                         if os.path.isdir(os.path.join(pdir, s)) and study_re.match(s))
    return total

def list_images(part, split):
    paths = []
    for ext in IMG_EXTS:
        paths.extend(glob.glob(os.path.join(base_dir, split, part, "**", ext), recursive=True))
    return paths

def label_from_path(p):
    pl = p.lower().replace("\\","/")
    if "positive" in pl: return 1
    if "negative" in pl: return 0
    return None

def count_images(part, split):
    imgs = list_images(part, split)
    pos = sum(1 for f in imgs if label_from_path(f)==1)
    neg = sum(1 for f in imgs if label_from_path(f)==0)
    return pos, neg, len(imgs)

print("Summary by part:")
for part in body_parts:
    pats_tr = count_patients(part, "train")
    studs_tr = count_studies(part, "train")
    pos_tr, neg_tr, img_tr = count_images(part, "train")

    pats_va = count_patients(part, "valid")
    studs_va = count_studies(part, "valid")
    pos_va, neg_va, img_va = count_images(part, "valid")

    print(f"{part:12s} | patients: train={pats_tr:4d} valid={pats_va:4d} | "
          f"studies: train={studs_tr:4d} valid={studs_va:4d} | "
          f"images: train={img_tr:5d} (pos={pos_tr},neg={neg_tr})  "
          f"valid={img_va:5d} (pos={pos_va},neg={neg_va})")


Summary by part:
XR_ELBOW     | patients: train=1711 valid= 152 | studies: train=1754 valid= 158 | images: train= 4931 (pos=2006,neg=2925)  valid=  465 (pos=230,neg=235)
XR_FINGER    | patients: train=1865 valid= 166 | studies: train=1935 valid= 175 | images: train= 5106 (pos=1968,neg=3138)  valid=  461 (pos=247,neg=214)
XR_FOREARM   | patients: train= 865 valid= 129 | studies: train= 877 valid= 133 | images: train= 1825 (pos=661,neg=1164)  valid=  301 (pos=151,neg=150)
XR_HUMERUS   | patients: train= 587 valid= 132 | studies: train= 592 valid= 135 | images: train= 1272 (pos=599,neg=673)  valid=  288 (pos=140,neg=148)
XR_SHOULDER  | patients: train=2694 valid= 173 | studies: train=2821 valid= 194 | images: train= 8379 (pos=4168,neg=4211)  valid=  563 (pos=278,neg=285)
XR_HAND      | patients: train=1945 valid= 159 | studies: train=2018 valid= 167 | images: train= 5543 (pos=1484,neg=4059)  valid=  460 (pos=189,neg=271)
XR_WRIST     | patients: train=3267 valid= 207 | studies: train=3460

In [ ]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

Device: cuda


#config Model

In [ ]:
import os, re, glob, gc, math, random
from typing import List, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision.models import resnet50, ResNet50_Weights, inception_v3, Inception_V3_Weights

from monai.data import Dataset
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, ScaleIntensityRanged,
    Resized, RandFlipd, RandRotated, Compose, EnsureTyped, Lambdad
)
from monai.metrics import ROCAUCMetric
from sklearn.metrics import (
    f1_score, accuracy_score, precision_recall_fscore_support,
    classification_report, confusion_matrix, roc_auc_score
)

In [ ]:
os.environ.setdefault("OMP_NUM_THREADS", "1")
torch.set_num_threads(1)
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

SEED = 1337
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_ROOT = "/content/data/MURA-v1.1"
BODY_PARTS = ["XR_ELBOW","XR_FINGER","XR_FOREARM","XR_HUMERUS","XR_SHOULDER","XR_HAND","XR_WRIST"]

IMG_SIZE_RESNET    = 320
IMG_SIZE_INCEPTION = 299

BATCH_RESNET    = 24
BATCH_INCEPTION = 10

ACC_STEPS_INCEPTION = 2

EPOCHS       = 20
LR           = 1e-4
WEIGHT_DECAY = 1e-4
PATIENCE     = 4
FREEZE_EPOCHS = 3
NUM_WORKERS   = 0


In [ ]:
def to_3ch(x: np.ndarray):
    if x.ndim == 2:
        x = x[None, ...]
    c = x.shape[0]
    if c == 1:
        return np.repeat(x, 3, axis=0)
    if c >= 3:
        return x[:3, ...]
    return np.concatenate([x, x[:1, ...]], axis=0)

In [ ]:
def collect_items(split_dir: str, part: str) -> List[Dict]:
    items = []
    for f in glob.glob(os.path.join(split_dir, part, "*", "*", "*")):
        fl = f.lower()
        if fl.endswith((".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff")):
            y = 1 if "positive" in fl else 0
            items.append({"img": f, "label": y})
    return items

In [ ]:
def build_sampler(train_items: List[Dict]) -> WeightedRandomSampler:
    labels = [it["label"] for it in train_items]
    cnt0 = labels.count(0); cnt1 = labels.count(1); total = len(labels)
    w0 = total / (2.0 * max(cnt0, 1))
    w1 = total / (2.0 * max(cnt1, 1))
    weights = [w1 if l == 1 else w0 for l in labels]
    return WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

In [ ]:
def build_transforms(img_size: int):
    IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)[:, None, None]
    IMAGENET_STD  = np.array([0.229, 0.224, 0.225], dtype=np.float32)[:, None, None]

    train_tfms = Compose([
        LoadImaged(keys=["img"], image_only=True),
        EnsureChannelFirstd(keys=["img"]),
        Lambdad(keys=["img"], func=to_3ch),
        ScaleIntensityRanged(keys=["img"], a_min=0, a_max=255, b_min=0.0, b_max=1.0, clip=True),
        Resized(keys=["img"], spatial_size=(img_size, img_size)),
        RandFlipd(keys=["img"], spatial_axis=1, prob=0.5),
        RandRotated(keys=["img"], range_x=np.deg2rad(7), prob=0.5, keep_size=True),
        Lambdad(keys=["img"], func=lambda x, m=IMAGENET_MEAN, s=IMAGENET_STD: (x.astype(np.float32) - m) / s),
        EnsureTyped(keys=["img"], dtype=np.float32),
    ])

    valid_tfms = Compose([
        LoadImaged(keys=["img"], image_only=True),
        EnsureChannelFirstd(keys=["img"]),
        Lambdad(keys=["img"], func=to_3ch),
        ScaleIntensityRanged(keys=["img"], a_min=0, a_max=255, b_min=0.0, b_max=1.0, clip=True),
        Resized(keys=["img"], spatial_size=(img_size, img_size)),
        Lambdad(keys=["img"], func=lambda x, m=IMAGENET_MEAN, s=IMAGENET_STD: (x.astype(np.float32) - m) / s),
        EnsureTyped(keys=["img"], dtype=np.float32),
    ])
    return train_tfms, valid_tfms

In [ ]:
def build_loaders(train_items, valid_items, img_size, batch_size, sampler):
    train_tfms, valid_tfms = build_transforms(img_size)
    train_ds = Dataset(train_items, transform=train_tfms)
    valid_ds = Dataset(valid_items, transform=valid_tfms)

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        sampler=sampler,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=False,
        prefetch_factor=None,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=False,
        prefetch_factor=None,
        drop_last=False,
    )
    return train_loader, valid_loader

In [ ]:
def build_test_loader(test_items, img_size, batch_size):
    _, valid_tfms = build_transforms(img_size)
    test_ds = Dataset(test_items, transform=valid_tfms)
    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=False,
        prefetch_factor=None,
    )
    return test_loader

In [ ]:
def inception_loss_and_logits(model_out, y, criterion):
    if hasattr(model_out, "logits") and hasattr(model_out, "aux_logits"):
        main_logits = model_out.logits
        aux_logits  = model_out.aux_logits
        loss = criterion(main_logits, y) + 0.3 * criterion(aux_logits, y)
        return main_logits, loss
    return model_out, criterion(model_out, y)

In [ ]:
def get_logits(model_out):
    return model_out.logits if hasattr(model_out, "logits") else model_out

In [ ]:
@torch.inference_mode()
def evaluate(model: nn.Module, loader: DataLoader, criterion: nn.Module, device: str):
    model.eval()
    auc_metric = ROCAUCMetric()
    total_loss = 0.0
    all_preds, all_labels = [], []

    for batch in loader:
        x = batch["img"].to(device, non_blocking=True).to(memory_format=torch.channels_last)
        y = batch["label"].long().to(device, non_blocking=True)

        out = model(x)
        logits = get_logits(out)
        loss = criterion(logits, y)
        total_loss += loss.item() * x.size(0)

        y_prob = torch.softmax(logits, dim=1)
        y_onehot = torch.nn.functional.one_hot(y, num_classes=2)
        auc_metric(y_pred=y_prob, y=y_onehot)

        preds = (y_prob[:, 1] >= 0.5).long()
        all_preds += preds.detach().cpu().tolist()
        all_labels += y.detach().cpu().tolist()

    auc = auc_metric.aggregate().item()
    auc_metric.reset()

    f1 = f1_score(all_labels, all_preds)
    acc = accuracy_score(all_labels, all_preds)
    loss_avg = total_loss / len(loader.dataset)
    return {"loss": loss_avg, "auc": auc, "f1": f1, "acc": acc}

In [ ]:
@torch.inference_mode()
def evaluate_split(model: nn.Module, loader: DataLoader, device: str) -> dict:
    model.eval()
    y_true, y_prob1, y_pred = [], [], []

    for batch in loader:
        x = batch["img"].to(device, non_blocking=True).to(memory_format=torch.channels_last)
        y = batch["label"].long().to(device, non_blocking=True)
        logits = get_logits(model(x))
        prob = torch.softmax(logits, dim=1)[:, 1]
        pred = (prob >= 0.5).long()

        y_true += y.detach().cpu().tolist()
        y_prob1 += prob.detach().cpu().tolist()
        y_pred  += pred.detach().cpu().tolist()

    acc = accuracy_score(y_true, y_pred)
    p, r, f1, support = precision_recall_fscore_support(y_true, y_pred, labels=[0,1], zero_division=0)
    macro_p = p.mean(); macro_r = r.mean(); macro_f1 = f1.mean()
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    try:
        auc = roc_auc_score(y_true, y_prob1)
    except:
        auc = float("nan")

    report = classification_report(y_true, y_pred, labels=[0,1], target_names=["neg","pos"], zero_division=0, output_dict=True)

    return {
        "accuracy": acc,
        "auc": auc,
        "per_class": {"precision": p.tolist(), "recall": r.tolist(), "f1": f1.tolist(), "support": support.tolist()},
        "macro": {"precision": macro_p, "recall": macro_r, "f1": macro_f1},
        "cm": cm.tolist(),
        "report_dict": report
    }

In [ ]:
def plot_curves(history_csv_path: str, out_dir: str):
    os.makedirs(out_dir, exist_ok=True)
    hist = pd.read_csv(history_csv_path)

    # Loss
    plt.figure()
    plt.plot(hist["epoch"], hist["train_loss"], label="Train Loss")
    plt.plot(hist["epoch"], hist["val_loss"], label="Val Loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Train/Val Loss per Epoch"); plt.legend()
    fig1 = os.path.join(out_dir, "loss_curve.png")
    plt.savefig(fig1, bbox_inches="tight"); plt.close()

    # AUC
    plt.figure()
    plt.plot(hist["epoch"], hist["val_auc"], label="Val AUC")
    plt.xlabel("Epoch"); plt.ylabel("AUC"); plt.title("Validation AUC per Epoch"); plt.legend()
    fig2 = os.path.join(out_dir, "val_auc_curve.png")
    plt.savefig(fig2, bbox_inches="tight"); plt.close()

    print(f"Saved plots:\n - {fig1}\n - {fig2}")


In [ ]:
def export_results_excel(row_dict: dict, out_path: str):
    df = pd.DataFrame([row_dict])
    df.to_excel(out_path, index=False)
    print(f"Exported Excel → {out_path}")

In [ ]:
def append_row_to_google_sheet(ws, row_dict: dict):
    if ws is None:
        return
    try:
        existing = ws.get_all_records()
        if len(existing) == 0:
            ws.append_row(list(row_dict.keys()))
        header = ws.row_values(1)
        values = [row_dict.get(col, "") for col in header]
        ws.append_row(values)
        print("Appended to Google Sheet.")
    except Exception as e:
        print("Google Sheet append error:", e)

In [ ]:
def build_resnet50(num_classes=2, pretrained=True) -> nn.Module:
    weights = ResNet50_Weights.IMAGENET1K_V2 if pretrained else None
    m = resnet50(weights=weights)
    m.fc = nn.Linear(m.fc.in_features, num_classes)
    return m

In [ ]:
def build_inception_v3(num_classes=2, pretrained=True, use_aux=True) -> nn.Module:
    weights = Inception_V3_Weights.IMAGENET1K_V1 if pretrained else None
    m = inception_v3(weights=weights, aux_logits=use_aux)
    m.fc = nn.Linear(m.fc.in_features, num_classes)
    if use_aux and hasattr(m, "AuxLogits") and hasattr(m.AuxLogits, "fc"):
        m.AuxLogits.fc = nn.Linear(m.AuxLogits.fc.in_features, num_classes)
    return m

In [ ]:
def _train_loop_generic(part, model, img_size, batch_size, use_inception_aux=False, acc_steps=1, out_base_dir="/content/ckpt", ws=None):
    os.makedirs(out_base_dir, exist_ok=True)
    out_dir = os.path.join(out_base_dir, part); os.makedirs(out_dir, exist_ok=True)

    train_items = collect_items(os.path.join(DATA_ROOT, "train"), part)
    valid_items = collect_items(os.path.join(DATA_ROOT, "valid"), part)
    test_items  = collect_items(os.path.join(DATA_ROOT, "test"),  part)

    print(f"{part} | Train: {len(train_items)} | Valid: {len(valid_items)} | Test: {len(test_items)}")
    if len(train_items) == 0 or len(valid_items) == 0:
        print("Bỏ qua: thiếu dữ liệu train/valid.")
        return None

    sampler = build_sampler(train_items)
    train_loader, valid_loader = build_loaders(train_items, valid_items, img_size, batch_size, sampler)
    test_loader = build_test_loader(test_items, img_size, batch_size) if len(test_items) > 0 else None

    model = model.to(DEVICE).to(memory_format=torch.channels_last)
    # backbone params để freeze
    exclude = ("fc.", "AuxLogits.") if use_inception_aux else ("fc.",)
    backbone_params = [p for n,p in model.named_parameters() if not any(n.startswith(e) for e in exclude)]

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.amp.GradScaler('cuda')

    if FREEZE_EPOCHS > 0:
        for p in backbone_params: p.requires_grad = False

    best_auc, best_epoch, no_improve = -1.0, -1, 0
    history = []

    model_name = type(model).__name__.lower()
    ckpt_path = os.path.join(out_dir, f"{part}_{model_name}_best.pth")

    from tqdm.auto import tqdm

    for epoch in range(1, EPOCHS + 1):
        model.train()
        running = 0.0
        optimizer.zero_grad(set_to_none=True)

        if epoch == FREEZE_EPOCHS + 1 and FREEZE_EPOCHS > 0:
            for p in backbone_params: p.requires_grad = True

        pbar = tqdm(train_loader, desc=f"{part}|{model_name} Ep {epoch}/{EPOCHS}")
        accum = 0

        for batch in pbar:
            x = batch["img"].to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
            y = batch["label"].long().to(DEVICE, non_blocking=True)

            with torch.amp.autocast('cuda', enabled=(DEVICE == "cuda")):
                out = model(x)
                if use_inception_aux:
                    logits, loss = inception_loss_and_logits(out, y, criterion)
                else:
                    logits = get_logits(out)
                    loss = criterion(logits, y)

                if acc_steps > 1:
                    loss = loss / acc_steps

            scaler.scale(loss).backward()
            accum += 1

            # update mỗi acc_steps
            if (acc_steps == 1) or (accum % acc_steps == 0):
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)

            running += (loss.item() * (acc_steps if acc_steps > 1 else 1)) * x.size(0)

        # Nếu batch không chia hết acc_steps
        if acc_steps > 1 and (accum % acc_steps != 0):
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

        train_loss = running / len(train_loader.dataset)
        metrics = evaluate(model, valid_loader, criterion, DEVICE)
        scheduler.step()

        history.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "val_loss": metrics["loss"],
            "val_auc": metrics["auc"],
            "val_f1": metrics["f1"],
            "val_acc": metrics["acc"]
        })

        print(f"Ep{epoch:02d} | train={train_loss:.4f} | val={metrics['loss']:.4f} | AUC={metrics['auc']:.4f} | F1={metrics['f1']:.4f} | Acc={metrics['acc']:.4f}")

        cur_auc = metrics["auc"] if not math.isnan(metrics["auc"]) else -1.0
        if cur_auc > best_auc:
            best_auc, best_epoch = cur_auc, epoch
            torch.save({
                "state_dict": model.state_dict(),
                "auc": best_auc,
                "epoch": epoch,
                "model_name": model_name,
                "part": part,
                "img_size": img_size
            }, ckpt_path)
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= PATIENCE:
                print(f"Early stopping at epoch {epoch}. Best AUC={best_auc:.4f} (epoch {best_epoch})")
                break

    # Lưu lịch sử + đồ thị
    hist_df = pd.DataFrame(history)
    hist_csv = os.path.join(out_dir, f"{part}_{model_name}_history.csv")
    hist_df.to_csv(hist_csv, index=False)
    print(f"Saved history → {hist_csv}")
    plot_curves(hist_csv, out_dir)

    # Load best và đánh giá chi tiết
    best = torch.load(ckpt_path, map_location=DEVICE)
    model.load_state_dict(best["state_dict"])
    model.eval()

    val_detail = evaluate_split(model, valid_loader, DEVICE)
    test_detail = evaluate_split(model, test_loader, DEVICE) if test_loader is not None else None

    out_xlsx = os.path.join(out_dir, f"{part}_{model_name}_metrics.xlsx")
    row = {
        "Part": part,
        "Model": model_name,
        "BestEpoch": best_epoch,
        "BestValAUC": round(best_auc, 6),
        "Val_Accuracy": round(val_detail["accuracy"], 6),
        "Val_Precision_0": round(val_detail["per_class"]["precision"][0], 6),
        "Val_Recall_0": round(val_detail["per_class"]["recall"][0], 6),
        "Val_F1_0": round(val_detail["per_class"]["f1"][0], 6),
        "Val_Support_0": val_detail["per_class"]["support"][0],
        "Val_Precision_1": round(val_detail["per_class"]["precision"][1], 6),
        "Val_Recall_1": round(val_detail["per_class"]["recall"][1], 6),
        "Val_F1_1": round(val_detail["per_class"]["f1"][1], 6),
        "Val_Support_1": val_detail["per_class"]["support"][1],
        "Val_Macro_P": round(val_detail["macro"]["precision"], 6),
        "Val_Macro_R": round(val_detail["macro"]["recall"], 6),
        "Val_Macro_F1": round(val_detail["macro"]["f1"], 6),
        "Val_AUC": round(val_detail["auc"], 6) if val_detail["auc"]==val_detail["auc"] else "",
    }

    if test_detail is not None:
        row.update({
            "Test_Accuracy": round(test_detail["accuracy"], 6),
            "Test_Macro_P": round(test_detail["macro"]["precision"], 6),
            "Test_Macro_R": round(test_detail["macro"]["recall"], 6),
            "Test_Macro_F1": round(test_detail["macro"]["f1"], 6),
            "Test_AUC": round(test_detail["auc"], 6) if test_detail["auc"]==test_detail["auc"] else "",
            "Test_Precision_0": round(test_detail["per_class"]["precision"][0], 6),
            "Test_Recall_0": round(test_detail["per_class"]["recall"][0], 6),
            "Test_F1_0": round(test_detail["per_class"]["f1"][0], 6),
            "Test_Support_0": test_detail["per_class"]["support"][0],
            "Test_Precision_1": round(test_detail["per_class"]["precision"][1], 6),
            "Test_Recall_1": round(test_detail["per_class"]["recall"][1], 6),
            "Test_F1_1": round(test_detail["per_class"]["f1"][1], 6),
            "Test_Support_1": test_detail["per_class"]["support"][1],
        })
    export_results_excel(row, out_xlsx)
    append_row_to_google_sheet(ws, row)

    # Giải phóng
    del model, optimizer, scheduler, train_loader, valid_loader, test_loader
    gc.collect()
    if DEVICE == "cuda":
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        try: torch.cuda.ipc_collect()
        except: pass

    return {"part": part, "best_auc": best_auc, "best_epoch": best_epoch, "ckpt": ckpt_path,
            "history_csv": hist_csv, "val_detail": val_detail, "test_detail": test_detail, "out_dir": out_dir}

In [ ]:
def train_one_part_resnet(part: str, out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_mura_resnet", ws=None):
    model = build_resnet50()
    return _train_loop_generic(part, model, IMG_SIZE_RESNET, BATCH_RESNET, use_inception_aux=False, acc_steps=1,
                               out_base_dir=out_base_dir, ws=ws)

In [ ]:
def train_one_part_inception(part: str, out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_inception", ws=None):
    model = build_inception_v3(num_classes=2, pretrained=True, use_aux=True)
    return _train_loop_generic(part, model, IMG_SIZE_INCEPTION, BATCH_INCEPTION, use_inception_aux=True,
                               acc_steps=ACC_STEPS_INCEPTION, out_base_dir=out_base_dir, ws=ws)

#**Resnet**

In [ ]:
BODY_PARTS = ["XR_FINGER","XR_FOREARM","XR_HUMERUS","XR_SHOULDER","XR_HAND","XR_WRIST"]

In [ ]:
if __name__ == "__main__":
    result = train_one_part_resnet("XR_ELBOW", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_mura_resnet")

XR_ELBOW | Train: 4931 | Valid: 465 | Test: 0


XR_ELBOW|resnet Ep 1/20:   0%|          | 0/206 [00:00<?, ?it/s]

Ep01 | train=0.6697 | val=0.6464 | AUC=0.7464 | F1=0.6667 | Acc=0.6667


XR_ELBOW|resnet Ep 2/20:   0%|          | 0/206 [00:00<?, ?it/s]

Ep02 | train=0.6420 | val=0.6213 | AUC=0.7745 | F1=0.6634 | Acc=0.7054


...

Ep07 | train=0.1983 | val=0.5290 | AUC=0.8852 | F1=0.8148 | Acc=0.8172


XR_ELBOW|resnet Ep 8/20:   0%|          | 0/206 [00:00<?, ?it/s]

Ep08 | train=0.1751 | val=0.6265 | AUC=0.8994 | F1=0.8274 | Acc=0.8430


XR_ELBOW|resnet Ep 9/20:   0%|          | 0/206 [00:00<?, ?it/s]

Ep09 | train=0.1409 | val=0.6965 | AUC=0.8881 | F1=0.8057 | Acc=0.7957
Early stopping at epoch 9. Best AUC=0.9036 (epoch 5)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_ELBOW/XR_ELBOW_resnet_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_ELBOW/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_ELBOW/val_auc_curve.png
✅ Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_ELBOW/XR_ELBOW_resnet_metrics.xlsx
Appended to Google Sheet.


In [ ]:
if __name__ == "__main__":
    result = train_one_part_resnet("XR_FINGER", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_mura_resnet")

XR_FINGER | Train: 5106 | Valid: 461 | Test: 0


XR_FINGER|resnet Ep 1/20:   0%|          | 0/213 [00:00<?, ?it/s]

Ep01 | train=0.6402 | val=0.6025 | AUC=0.7510 | F1=0.7054 | Acc=0.6811


XR_FINGER|resnet Ep 2/20:   0%|          | 0/213 [00:00<?, ?it/s]

Ep02 | train=0.6063 | val=0.5919 | AUC=0.7676 | F1=0.7094 | Acc=0.7050


...

Ep05 | train=0.3501 | val=0.6625 | AUC=0.8011 | F1=0.7647 | Acc=0.7397


XR_FINGER|resnet Ep 6/20:   0%|          | 0/213 [00:00<?, ?it/s]

Ep06 | train=0.2870 | val=0.6567 | AUC=0.8284 | F1=0.7402 | Acc=0.7549


XR_FINGER|resnet Ep 7/20:   0%|          | 0/213 [00:00<?, ?it/s]

Ep07 | train=0.2255 | val=1.1165 | AUC=0.7277 | F1=0.6741 | Acc=0.6833
Early stopping at epoch 7. Best AUC=0.8304 (epoch 3)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_FINGER/XR_FINGER_resnet_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_FINGER/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_FINGER/val_auc_curve.png
✅ Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_FINGER/XR_FINGER_resnet_metrics.xlsx
Appended to Google Sheet.


In [ ]:
if __name__ == "__main__":
    result = train_one_part_resnet("XR_FOREARM", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_mura_resnet")

XR_FOREARM | Train: 1825 | Valid: 301 | Test: 0


XR_FOREARM|resnet Ep 1/20:   0%|          | 0/77 [00:00<?, ?it/s]

Ep01 | train=0.6764 | val=0.6699 | AUC=0.6710 | F1=0.6312 | Acc=0.6312


XR_FOREARM|resnet Ep 2/20:   0%|          | 0/77 [00:00<?, ?it/s]

Ep02 | train=0.6483 | val=0.6518 | AUC=0.7401 | F1=0.4622 | Acc=0.5980


...

Ep10 | train=0.1033 | val=1.0744 | AUC=0.8759 | F1=0.7470 | Acc=0.7907


XR_FOREARM|resnet Ep 11/20:   0%|          | 0/77 [00:00<?, ?it/s]

Ep11 | train=0.1121 | val=0.7617 | AUC=0.8679 | F1=0.8000 | Acc=0.8140


XR_FOREARM|resnet Ep 12/20:   0%|          | 0/77 [00:00<?, ?it/s]

Ep12 | train=0.0702 | val=0.9155 | AUC=0.8720 | F1=0.8127 | Acc=0.8239
Early stopping at epoch 12. Best AUC=0.8875 (epoch 8)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_FOREARM/XR_FOREARM_resnet_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_FOREARM/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_FOREARM/val_auc_curve.png
✅ Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_FOREARM/XR_FOREARM_resnet_metrics.xlsx
Appended to Google Sheet.


In [ ]:
if __name__ == "__main__":
    result = train_one_part_resnet("XR_HUMERUS", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_mura_resnet")

XR_HUMERUS | Train: 1272 | Valid: 288 | Test: 0


XR_HUMERUS|resnet Ep 1/20:   0%|          | 0/53 [00:00<?, ?it/s]

Ep01 | train=0.6879 | val=0.6710 | AUC=0.6907 | F1=0.6338 | Acc=0.6389


XR_HUMERUS|resnet Ep 2/20:   0%|          | 0/53 [00:00<?, ?it/s]

Ep02 | train=0.6703 | val=0.6493 | AUC=0.8068 | F1=0.7582 | Acc=0.7188


...

Ep18 | train=0.0219 | val=0.7667 | AUC=0.9210 | F1=0.8681 | Acc=0.8681


XR_HUMERUS|resnet Ep 19/20:   0%|          | 0/53 [00:00<?, ?it/s]

Ep19 | train=0.0162 | val=0.7621 | AUC=0.9216 | F1=0.8671 | Acc=0.8681


XR_HUMERUS|resnet Ep 20/20:   0%|          | 0/53 [00:00<?, ?it/s]

Ep20 | train=0.0157 | val=0.7651 | AUC=0.9210 | F1=0.8702 | Acc=0.8715
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_HUMERUS/XR_HUMERUS_resnet_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_HUMERUS/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_HUMERUS/val_auc_curve.png
✅ Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_HUMERUS/XR_HUMERUS_resnet_metrics.xlsx
Appended to Google Sheet.


In [ ]:
if __name__ == "__main__":
    result = train_one_part_resnet("XR_SHOULDER", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_mura_resnet")

XR_SHOULDER | Train: 8379 | Valid: 563 | Test: 0


XR_SHOULDER|resnet Ep 1/20:   0%|          | 0/350 [00:00<?, ?it/s]

Ep01 | train=0.6682 | val=0.6418 | AUC=0.7555 | F1=0.6937 | Acc=0.7052


XR_SHOULDER|resnet Ep 2/20:   0%|          | 0/350 [00:00<?, ?it/s]

Ep02 | train=0.6263 | val=0.6157 | AUC=0.7717 | F1=0.7194 | Acc=0.7105


...

Ep15 | train=0.0679 | val=1.5366 | AUC=0.8424 | F1=0.7584 | Acc=0.7691


XR_SHOULDER|resnet Ep 16/20:   0%|          | 0/350 [00:00<?, ?it/s]

Ep16 | train=0.0513 | val=1.5507 | AUC=0.8446 | F1=0.7707 | Acc=0.7833


XR_SHOULDER|resnet Ep 17/20:   0%|          | 0/350 [00:00<?, ?it/s]

Ep17 | train=0.0351 | val=1.6402 | AUC=0.8464 | F1=0.7689 | Acc=0.7833
Early stopping at epoch 17. Best AUC=0.8603 (epoch 13)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_SHOULDER/XR_SHOULDER_resnet_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_SHOULDER/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_SHOULDER/val_auc_curve.png
✅ Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_SHOULDER/XR_SHOULDER_resnet_metrics.xlsx
Appended to Google Sheet.


In [ ]:
if __name__ == "__main__":
    result = train_one_part_resnet("XR_HAND", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_mura_resnet")

XR_HAND | Train: 5543 | Valid: 460 | Test: 0


XR_HAND|resnet Ep 1/20:   0%|          | 0/231 [00:00<?, ?it/s]

Ep01 | train=0.6750 | val=0.6667 | AUC=0.7452 | F1=0.6383 | Acc=0.5935


XR_HAND|resnet Ep 2/20:   0%|          | 0/231 [00:00<?, ?it/s]

Ep02 | train=0.6489 | val=0.6299 | AUC=0.7774 | F1=0.6539 | Acc=0.6848


...

Ep05 | train=0.3679 | val=0.6053 | AUC=0.8150 | F1=0.7065 | Acc=0.7652


XR_HAND|resnet Ep 6/20:   0%|          | 0/231 [00:00<?, ?it/s]

Ep06 | train=0.2944 | val=0.8373 | AUC=0.7840 | F1=0.6603 | Acc=0.7674


XR_HAND|resnet Ep 7/20:   0%|          | 0/231 [00:00<?, ?it/s]

Ep07 | train=0.2484 | val=0.8598 | AUC=0.7801 | F1=0.6435 | Acc=0.7543
Early stopping at epoch 7. Best AUC=0.8162 (epoch 3)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_HAND/XR_HAND_resnet_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_HAND/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_HAND/val_auc_curve.png
✅ Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_HAND/XR_HAND_resnet_metrics.xlsx
Appended to Google Sheet.


In [ ]:
if __name__ == "__main__":
    result = train_one_part_resnet("XR_WRIST", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_mura_resnet")

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 217MB/s]


XR_WRIST | Train: 9752 | Valid: 659 | Test: 0


XR_WRIST|resnet Ep 1/20:   0%|          | 0/407 [00:00<?, ?it/s]

Ep01 | train=0.6472 | val=0.6151 | AUC=0.7235 | F1=0.5985 | Acc=0.6722


...

Ep13 | train=0.0751 | val=1.2321 | AUC=0.8877 | F1=0.8195 | Acc=0.8543


XR_WRIST|resnet Ep 14/20:   0%|          | 0/407 [00:00<?, ?it/s]

Ep14 | train=0.0746 | val=1.0892 | AUC=0.8985 | F1=0.8077 | Acc=0.8331


XR_WRIST|resnet Ep 15/20:   0%|          | 0/407 [00:00<?, ?it/s]

Ep15 | train=0.0488 | val=1.3152 | AUC=0.8963 | F1=0.8222 | Acc=0.8543
Early stopping at epoch 15. Best AUC=0.9081 (epoch 11)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_WRIST/XR_WRIST_resnet_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_WRIST/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_WRIST/val_auc_curve.png
✅ Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_mura_resnet/XR_WRIST/XR_WRIST_resnet_metrics.xlsx
Appended to Google Sheet.


#**Inception**

In [ ]:
if __name__ == "__main__":
    result = train_one_part_inception("XR_ELBOW", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_inception")


XR_ELBOW | Train: 4931 | Valid: 465 | Test: 0


XR_ELBOW|inception3 Ep 1/20:   0%|          | 0/493 [00:00<?, ?it/s]

Ep01 | train=0.8618 | val=0.6524 | AUC=0.7525 | F1=0.6864 | Acc=0.5892


XR_ELBOW|inception3 Ep 2/20:   0%|          | 0/493 [00:00<?, ?it/s]

Ep02 | train=0.8061 | val=0.6054 | AUC=0.8057 | F1=0.7177 | Acc=0.7462


...

Ep09 | train=0.2928 | val=0.5196 | AUC=0.9035 | F1=0.8315 | Acc=0.8344


XR_ELBOW|inception3 Ep 10/20:   0%|          | 0/493 [00:00<?, ?it/s]

Ep10 | train=0.2390 | val=0.5875 | AUC=0.9063 | F1=0.8081 | Acc=0.8172


XR_ELBOW|inception3 Ep 11/20:   0%|          | 0/493 [00:00<?, ?it/s]

Ep11 | train=0.2254 | val=0.5619 | AUC=0.9103 | F1=0.8319 | Acc=0.8366
Early stopping at epoch 11. Best AUC=0.9243 (epoch 7)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_ELBOW/XR_ELBOW_inception3_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_ELBOW/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_ELBOW/val_auc_curve.png
Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_ELBOW/XR_ELBOW_inception3_metrics.xlsx


In [ ]:
if __name__ == "__main__":
    result = train_one_part_inception("XR_FINGER", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_inception")

XR_FINGER | Train: 5106 | Valid: 461 | Test: 0


XR_FINGER|inception3 Ep 1/20:   0%|          | 0/510 [00:00<?, ?it/s]

Ep01 | train=0.8442 | val=0.6229 | AUC=0.7456 | F1=0.7008 | Acc=0.6703


XR_FINGER|inception3 Ep 2/20:   0%|          | 0/510 [00:00<?, ?it/s]

Ep02 | train=0.7782 | val=0.5983 | AUC=0.7616 | F1=0.7302 | Acc=0.6681


...

Ep07 | train=0.4194 | val=0.5611 | AUC=0.8504 | F1=0.7742 | Acc=0.7722


XR_FINGER|inception3 Ep 8/20:   0%|          | 0/510 [00:00<?, ?it/s]

Ep08 | train=0.3539 | val=0.7130 | AUC=0.8223 | F1=0.7591 | Acc=0.7549


XR_FINGER|inception3 Ep 9/20:   0%|          | 0/510 [00:00<?, ?it/s]

Ep09 | train=0.2839 | val=0.7245 | AUC=0.8419 | F1=0.7771 | Acc=0.7722
Early stopping at epoch 9. Best AUC=0.8522 (epoch 5)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_FINGER/XR_FINGER_inception3_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_FINGER/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_FINGER/val_auc_curve.png
Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_FINGER/XR_FINGER_inception3_metrics.xlsx


In [ ]:
if __name__ == "__main__":
    result = train_one_part_inception("XR_FOREARM", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_inception")


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [00:00<00:00, 167MB/s]


XR_FOREARM | Train: 1825 | Valid: 301 | Test: 0


XR_FOREARM|inception3 Ep 1/20:   0%|          | 0/182 [00:00<?, ?it/s]

Ep01 | train=0.8860 | val=0.6739 | AUC=0.6643 | F1=0.4155 | Acc=0.5980


...

Ep12 | train=0.1865 | val=0.8001 | AUC=0.8613 | F1=0.7481 | Acc=0.7807


XR_FOREARM|inception3 Ep 13/20:   0%|          | 0/182 [00:00<?, ?it/s]

Ep13 | train=0.1475 | val=0.9421 | AUC=0.8363 | F1=0.7518 | Acc=0.7741


XR_FOREARM|inception3 Ep 14/20:   0%|          | 0/182 [00:00<?, ?it/s]

Ep14 | train=0.0938 | val=1.0243 | AUC=0.8669 | F1=0.7698 | Acc=0.7973
Early stopping at epoch 14. Best AUC=0.8751 (epoch 10)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_FOREARM/XR_FOREARM_inception3_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_FOREARM/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_FOREARM/val_auc_curve.png
Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_FOREARM/XR_FOREARM_inception3_metrics.xlsx


In [ ]:
if __name__ == "__main__":
    result = train_one_part_inception("XR_HUMERUS", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_inception")

XR_HUMERUS | Train: 1272 | Valid: 288 | Test: 0


XR_HUMERUS|inception3 Ep 1/20:   0%|          | 0/127 [00:00<?, ?it/s]

Ep01 | train=0.8609 | val=0.6703 | AUC=0.6672 | F1=0.6606 | Acc=0.6146


XR_HUMERUS|inception3 Ep 2/20:   0%|          | 0/127 [00:00<?, ?it/s]

Ep02 | train=0.8235 | val=0.6436 | AUC=0.7507 | F1=0.6058 | Acc=0.6701


...

Ep10 | train=0.2111 | val=0.5567 | AUC=0.9034 | F1=0.8446 | Acc=0.8403


XR_HUMERUS|inception3 Ep 11/20:   0%|          | 0/127 [00:00<?, ?it/s]

Ep11 | train=0.1941 | val=0.5819 | AUC=0.9016 | F1=0.8720 | Acc=0.8715


XR_HUMERUS|inception3 Ep 12/20:   0%|          | 0/127 [00:00<?, ?it/s]

Ep12 | train=0.1339 | val=0.6148 | AUC=0.8994 | F1=0.8303 | Acc=0.8368
Early stopping at epoch 12. Best AUC=0.9139 (epoch 8)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_HUMERUS/XR_HUMERUS_inception3_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_HUMERUS/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_HUMERUS/val_auc_curve.png
Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_HUMERUS/XR_HUMERUS_inception3_metrics.xlsx


In [ ]:
if __name__ == "__main__":
    result = train_one_part_inception("XR_SHOULDER", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_inception")

XR_SHOULDER | Train: 8379 | Valid: 563 | Test: 0


XR_SHOULDER|inception3 Ep 1/20:   0%|          | 0/837 [00:00<?, ?it/s]

Ep01 | train=0.8690 | val=0.6595 | AUC=0.6896 | F1=0.5920 | Acc=0.6377


XR_SHOULDER|inception3 Ep 2/20:   0%|          | 0/837 [00:00<?, ?it/s]

Ep02 | train=0.8296 | val=0.6387 | AUC=0.7229 | F1=0.6701 | Acc=0.6590


...

Ep08 | train=0.4404 | val=0.6029 | AUC=0.8295 | F1=0.7294 | Acc=0.7496


XR_SHOULDER|inception3 Ep 9/20:   0%|          | 0/837 [00:00<?, ?it/s]

Ep09 | train=0.3309 | val=0.7018 | AUC=0.8382 | F1=0.7664 | Acc=0.7726


XR_SHOULDER|inception3 Ep 10/20:   0%|          | 0/837 [00:00<?, ?it/s]

Ep10 | train=0.3197 | val=0.7017 | AUC=0.8331 | F1=0.7634 | Acc=0.7567
Early stopping at epoch 10. Best AUC=0.8482 (epoch 6)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_SHOULDER/XR_SHOULDER_inception3_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_SHOULDER/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_SHOULDER/val_auc_curve.png
Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_SHOULDER/XR_SHOULDER_inception3_metrics.xlsx


In [ ]:
if __name__ == "__main__":
    result = train_one_part_inception("XR_HAND", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_inception")

XR_HAND | Train: 5543 | Valid: 460 | Test: 0


XR_HAND|inception3 Ep 1/20:   0%|          | 0/554 [00:00<?, ?it/s]

Ep01 | train=0.8837 | val=0.6692 | AUC=0.6206 | F1=0.4796 | Acc=0.5848


XR_HAND|inception3 Ep 2/20:   0%|          | 0/554 [00:00<?, ?it/s]

Ep02 | train=0.8391 | val=0.6577 | AUC=0.6487 | F1=0.5348 | Acc=0.6217


...

Ep07 | train=0.4683 | val=0.6203 | AUC=0.8090 | F1=0.6885 | Acc=0.7522


XR_HAND|inception3 Ep 8/20:   0%|          | 0/554 [00:00<?, ?it/s]

Ep08 | train=0.3979 | val=0.6541 | AUC=0.8142 | F1=0.7030 | Acc=0.7630


XR_HAND|inception3 Ep 9/20:   0%|          | 0/554 [00:00<?, ?it/s]

Ep09 | train=0.3061 | val=0.8284 | AUC=0.8056 | F1=0.6787 | Acc=0.7674
Early stopping at epoch 9. Best AUC=0.8515 (epoch 5)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_HAND/XR_HAND_inception3_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_HAND/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_HAND/val_auc_curve.png
Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_HAND/XR_HAND_inception3_metrics.xlsx


In [ ]:
if __name__ == "__main__":
    result = train_one_part_inception("XR_WRIST", out_base_dir="/content/drive/MyDrive/DeepLearning/ckpt_inception")

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [00:00<00:00, 156MB/s] 


XR_WRIST | Train: 9752 | Valid: 659 | Test: 0


XR_WRIST|inception3 Ep 1/20:   0%|          | 0/975 [00:00<?, ?it/s]

Ep01 | train=0.8223 | val=0.6244 | AUC=0.7319 | F1=0.6418 | Acc=0.6646


...

Ep12 | train=0.1854 | val=0.7424 | AUC=0.8862 | F1=0.8131 | Acc=0.8361


XR_WRIST|inception3 Ep 13/20:   0%|          | 0/975 [00:00<?, ?it/s]

Ep13 | train=0.1378 | val=0.9837 | AUC=0.8955 | F1=0.8137 | Acc=0.8437


XR_WRIST|inception3 Ep 14/20:   0%|          | 0/975 [00:00<?, ?it/s]

Ep14 | train=0.1457 | val=0.9543 | AUC=0.8941 | F1=0.8096 | Acc=0.8437
Early stopping at epoch 14. Best AUC=0.8989 (epoch 10)
✅ Saved history → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_WRIST/XR_WRIST_inception3_history.csv
Saved plots:
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_WRIST/loss_curve.png
 - /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_WRIST/val_auc_curve.png
Exported Excel → /content/drive/MyDrive/DeepLearning/ckpt_inception/XR_WRIST/XR_WRIST_inception3_metrics.xlsx


In [ ]:
BODY_PARTS = ["XR_ELBOW","XR_FINGER","XR_FOREARM","XR_HUMERUS","XR_SHOULDER","XR_HAND","XR_WRIST"]